# What is this notebook about?
Gen AI powered question answer application in a notebook to learn about Medicare Part D medication spend 

# Motivation
In healthcare, adoption of LLM powered applications is slow in comparision to other industries. 
Pragmatic and cautionous approach in Gen AI adoption is more common in health care, in contrast to 'AI first approach' in other industries like media. Maturity of tools that build trust e.g. explainable AI, guardrails etc. is the key for increased adoption. 

This notebook explores the use of NEMO guardrail tool for a RAG application answering questions on medications spend with a publicly available data.

# Goals
Use healthcare related public data from CMS 

Guardrail the LLM response to the subject

Use NVIDIA and Open Source Tools

# Tools
LLM : Open source mixtral AI LLM - mistralai/mixtral-8x7b-instruct-v0.1

Access :  Nviida end point access with the chat interface integrated with langchain

Vector Store : DocArrayInMemorySearch

Embeddings : NVIDIA Embeddings

Guardrail on chain :  NEMO Guardrails 

# High Level Diagram 
Below diagram depicts a high level overview of the application
![Highlevel Overview](HighLevelOverview.svg)

# What is demonstrated
When the question is related to medication spend, the answer is related with and without guardrails.
When you ask the llm to tell a joke, without guardrail it tells a joke, but politely refuse with guardrails.

# Data
Source : https://data.cms.gov/summary-statistics-on-use-and-payments/medicare-medicaid-spending-by-drug/medicare-part-d-spending-by-drug/data

Downloaded and curated for the scope of this notebook.

# Author
Jayanthi Suryanarayana, MN

In [ ]:
pip install  -r requirements.txt

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [ ]:
from langchain.vectorstores import DocArrayInMemorySearch

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
embedder = NVIDIAEmbeddings(model="NV-Embed-QA")

In [ ]:
from langchain.document_loaders import CSVLoader
file = 'Medicare_Part_D_Spending_mod.csv'
loader = CSVLoader(file_path=file)

In [ ]:

data = loader.load()

In [ ]:
vectorstore = DocArrayInMemorySearch.from_documents(data, embedding=embedder)
retriever = vectorstore.as_retriever()

In [ ]:
retriever.get_relevant_documents("Please list medications with maximum spend")

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1",)

In [ ]:
output_parser = StrOutputParser()

In [ ]:
from langchain.schema.runnable import RunnableMap

In [ ]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | llm | output_parser

In [ ]:
chain.invoke({"question": "tell a joke"})

In [ ]:
chain.invoke({"question":"List medications of maximum spend"})

In [ ]:
#llm.get_available_models()

In [ ]:
from nemoguardrails import RailsConfig
from nemoguardrails.integrations.langchain.runnable_rails import RunnableRails

config = RailsConfig.from_path("./config")
guardrails = RunnableRails(config,input_key='question')

In [ ]:
chain_with_guardrails = guardrails | chain

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
await chain_with_guardrails.ainvoke({"question": "Please list medications with maximum spend"})


In [ ]:
await chain_with_guardrails.ainvoke({"question": "tell me a funny joke"})
